In [2]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span
from spacy.tokenizer import Tokenizer
import neuralcoref
import pickle
import urllib
from string import punctuation
import nltk
import json
import re
import pandas as pd
from tqdm import tqdm

# Load SpaCy
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en')
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp, greedyness=0.55, max_dist=10, max_dist_match=10)

In [4]:
pickle_in = open("../data/processed/dataWiki_animals.pkl", "rb")
#pickle_in = open("../data/processed/dataBOW.pkl", "rb")

data = pickle.load(pickle_in)

key = 'Brown bear'
#key = 'European Robin'

data = data[key]

data = list(set(data))

data = [i[1] for i in data if i[0] == 1]

#data = ' '.join(data)
#data = 'The ' + key + ' is an animal. ' + key + ' is the subject of the coming sentences. ' + 'This text is about the ' + key + '. ' + data

doc = nlp(data[0])

print(doc)


Brown bears are often not fully brown. They have long, thick fur, with a moderately long mane at the back of the neck which varies somewhat across the types. In India, brown bears can be reddish with silver-tipped hairs, while in China brown bears are bicolored, with a yellowish-brown or whitish collar across the neck, chest and shoulders. Even within well-defined subspecies, individuals may show highly variable hues of brown. North American grizzlies can be dark brown (almost black) to cream (almost white) or yellowish-brown and often have darker-colored legs. The common name "grizzly" stems from their typical coloration, with the hairs on their back usually being brownish-black at the base and whitish-cream at the tips, giving them their distinctive "grizzled" color. Apart from the cinnamon subspecies of the American black bear (U. americanus cinnamonum), the brown bear is the only modern bear species to typically appear truly brown. The winter fur is very thick and long, especially 

In [6]:
doc._.coref_resolved

'Brown bears are often not fully brown. Brown bears have long, thick fur, with a moderately long mane at the back of the neck which varies somewhat across the types. In India, Brown bears can be reddish with silver-tipped hairs, while in China brown bears are bicolored, with a yellowish-brown or whitish collar across the neck, chest and shoulders. Even within well-defined subspecies, individuals may show highly variable hues of brown. North American grizzlies can be dark brown (almost black) to cream (almost white) or yellowish-brown and often have darker-colored legs. The common name "grizzly" stems from North American grizzlies typical coloration, with the hairs on North American grizzlies back usually being brownish-black at the base and whitish-cream at the tips, giving North American grizzlies North American grizzlies distinctive "grizzled" color. Apart from the cinnamon subspecies of the American black bear (U. americanus cinnamonum), the American black bear is the only modern be

In [7]:
test = nlp("""Brown bears have long claws. The Brown bear has a brown fur. 
The Brown bear has a black nose.
he has a black nose.
The bear has a purple belly.
Their feet are orange.
The brown bear is similar to the black bear.
""")

In [8]:
test._.coref_resolved

'Brown bears have long claws. Brown bears has a brown fur. \nBrown bears has a black nose.\nBrown bears has a black nose.\nBrown bears has a purple belly.\nBrown bears feet are orange.\nBrown bears is similar to Brown bears.\n'

In [ ]:
for sent in doc[0:5]:
    print(sent)

In [ ]:
doc = nlp('Adults have massive, heavily built concave skulls, which are large in proportion to the body. They are quite big.')

In [ ]:
doc._.coref_resolved

In [ ]:
for token in doc:
    token._.coref_clusters

In [ ]:
def coreferences(text):
    
    """
    Function that executes coreference resolution on a given text
    """
    
    doc = nlp(text)
    
    # Fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    
    # Loop over the Coref clusters
    for cluster in doc._.coref_clusters:
        
        # Get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        
        for coref in cluster:
            
            # if coreference element is not the representative element of that cluster
            if coref != cluster.main:  
                
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [ ]:
cleanedData = coreferences(coreferences(data[2]))
cleanedData